# Побудова оцінок та довірчих інтервалів

Будуємо довірчі інтервали для `n = 100, 10_000, 1_000_000`
Рівень довіри = `0,99` (рівень значущості = `0.01`

#### Довірчі інтервали для м.с. та дисперсії у випадку нормально розподіленої генеральної сукупності.
##### Перша частина леми 1. (Пункт А):
Вибіркове математичне сподівання: $\hat{a}_{n}=\frac {1}{n}\sum^{n}_{k=1}X_k$
Вибіркова дисперсія: $\hat{\sigma}^2_{n}=\frac {1}{n}\sum^{n}_{k=1}(X_k-\hat{a}_{n})^2$
Дов інтервал $a \in a_n \pm \frac{z_{\gamma}  \sigma_n}{\sqrt{n}}$
К-ть ітерацій: $n$

In [10]:
import math

import numpy as np
from scipy.stats import chi2, t, norm

In [11]:
def calc_sample_variance(samples, sample_mean: float):
    squared_variance = 0.
    for sample in samples:
        squared_variance += (sample - sample_mean) ** 2

    squared_variance /= (len(samples) - 1)
    return math.sqrt(squared_variance)


def calc_sample_mean(samples):
    mean = 0.
    for sample in samples:
        mean += sample
    return mean / len(samples)


def calc_s_squared(samples, sample_mean):
    s_squared = 0
    for sample in samples:
        s_squared += (sample - sample_mean) ** 2

    s_squared /= (len(samples) - 1)
    return s_squared


# generating samples with help of N(0,1) distribution
def gen_samples(size: int):
    samples = size * [None]
    for i in range(size):
        samples[i] = np.random.normal(0, 1)
    return samples

In [12]:
n_arr = [100, 10_000, 1_000_000]  # for calculating Confidence Intervals
m_arr = [1, 10, 100, 1_000, 10_000]  # for calculating Q_m

samples_list = len(n_arr) * [None]
alfa = 0.01
gama = 1 - alfa
z_gama = 2.575  # norm distro 0.99 quantile

Q_m = lambda q, n: np.sum(q) / n
# calculating variance in integral methods
# незміщена?
calc_variance = lambda q, n, Q: (1 / (n - 1)) * (np.sum([qi ** 2 for qi in q]) - n * Q ** 2)

eta = lambda x: 1 / x - 1
xi_i = lambda x: -1 * np.log(x)

# calculating 0.99 quantiles
print(f"samples_num_ci:{n_arr}\nm_arr:{m_arr}")

samples_num_ci:[100, 10000, 1000000]
m_arr:[1, 10, 100, 1000, 10000]


In [13]:
# calculating Confidence Interval for the EXPECTATION(sigma^2) under the assumption:
# - observed random value ~ N(0,1)
# - sample variance is unknown
def calculate_ci_1(samples):
    sample_mean = calc_sample_mean(samples)
    n_sq_root = math.sqrt(len(samples))
    s_squared = calc_s_squared(samples, sample_mean)

    g = t.ppf((1 + gama) / 2, df=len(samples) - 1)
    return [sample_mean - math.sqrt(s_squared) * g / n_sq_root, sample_mean + math.sqrt(s_squared) * g / n_sq_root]


# calculating Confidence Interval for the EXPECTATION under assumption:
# - observed value distribution is unknown
# asymptotic confidence interval
def calculate_ci_2(samples, gama):
    sample_mean = calc_sample_mean(samples)
    sample_variance = calc_sample_variance(samples, sample_mean)

    n_sq_root = math.sqrt(len(samples))

    c = norm.ppf((1 + gama) / 2)
    return [sample_mean - sample_variance * c / n_sq_root, sample_mean + sample_variance * c / n_sq_root]


# calculating Confidence Interval for the variance under assumption:
# - observed random value ~ N(0,1)
def calculate_ci_3(samples, gama):
    sample_mean = calc_sample_mean(samples)
    g_1 = chi2.ppf((1 - gama) / 2, df=len(samples) - 1)
    g_2 = chi2.ppf((1 + gama) / 2, df=len(samples) - 1)

    squared_difference = 0
    for sample in samples:
        squared_difference = (sample - sample_mean) ** 2

    return [squared_difference / g_2, squared_difference / g_1]

In [14]:
for i, sample_size in enumerate(n_arr):
    samples_list[i] = gen_samples(sample_size)
    sample_mean = calc_sample_mean(samples_list[i])
    sample_variance = calc_sample_variance(samples_list[i], sample_mean)

    ci_1 = calculate_ci_1(samples_list[i])
    print('\n[1] Довірчий інтервал для мат. спод. M при невідомій дисперсії')
    print(f'Кількість спостережень: n = {sample_size}')
    print(f'Вибіркове математичне сподівання: {sample_mean}')
    print(f'Довірчий інтервал:[{ci_1[0]} ; {ci_1[1]}]')
    print(f'Довжина довірчого інтервалу: {ci_1[1] - ci_1[0]}')

    ci_2 = calculate_ci_2(samples_list[i], gama)
    print('\n[2] Довірчий інтервал для мат. спод. M при невідомому розподілі')
    print(f'Кількість спостережень: n = {sample_size}')
    print(f'Вибіркове математичне сподівання: {sample_mean}')
    print(f'Довірчий інтервал:[{ci_2[0]} ; {ci_2[1]}]')
    print(f'Довжина довірчого інтервалу: {ci_2[1] - ci_2[0]}')

    ci_3 = calculate_ci_3(samples_list[i], gama)
    print('\n[3] Lовірчий інтервал для дисперсії')
    print(f'Кількість спостережень: n = {sample_size}')
    print(f'Вибіркова дисперсія = {sample_variance}')
    print(f'Довірчий інтервал:[{ci_3[0]} ; {ci_3[1]}]')
    print(f'Довжина довірчого інтервалу: {ci_3[1] - ci_3[0]}')


[1] Довірчий інтервал для мат. спод. M при невідомій дисперсії
Кількість спостережень: n = 100
Вибіркове математичне сподівання: -0.04259839099926314
Довірчий інтервал:[-0.2952052699945802 ; 0.2100084879960539]
Довжина довірчого інтервалу: 0.5052137579906341

[2] Довірчий інтервал для мат. спод. M при невідомому розподілі
Кількість спостережень: n = 100
Вибіркове математичне сподівання: -0.04259839099926314
Довірчий інтервал:[-0.29034087097785405 ; 0.2051440889793278]
Довжина довірчого інтервалу: 0.49548495995718184

[3] Lовірчий інтервал для дисперсії
Кількість спостережень: n = 100
Вибіркова дисперсія = 0.9617969623890014
Довірчий інтервал:[0.0005988133301462133 ; 0.0012513457656846269]
Довжина довірчого інтервалу: 0.0006525324355384135

[1] Довірчий інтервал для мат. спод. M при невідомій дисперсії
Кількість спостережень: n = 10000
Вибіркове математичне сподівання: 0.020098354942593568
Довірчий інтервал:[-0.005972714988537586 ; 0.04616942487372472]
Довжина довірчого інтервалу: 0.05

# Метод 1
Будемо обраховувати інтеграл за допомогою методу Монте-Карло:
$$Q_m = \bf{M}(I(\eta > \xi_{1} + \ldots + \xi_m))$$

(варто підмітити, що із збільшенням m значення інтеграла зменшується у $\sqrt{m}$ разів)

In [15]:
q_1 = lambda eta, samples: 1 if eta > np.sum(samples) else 0
m_arr = [1, 10, 100, 1_000]  # for calculating Q_m
n_0 = 5000  #

for m in m_arr:
    indicators = []

    for j in range(n_0):
        samples = np.random.random(m)
        indicators.append(q_1(eta(np.random.random(1)), samples))

    Q = Q_m(indicators, len(indicators))
    sigma = calc_variance(indicators, len(indicators), Q)

    print(f"Кількість зразків: {m}")
    print(f"Оцінка Q: {Q}")
    print(f"Вибіркова дисперсія: {sigma}")
    print(f"Асимптотичний довірчий інтервал: {calculate_ci_2(indicators, gama)}")
    print(f"Кількість спостережень: {n_0}")
    print(f"N*: {(2.575 * 100) ** 2 * sigma / (Q ** 2)}\n")

Кількість зразків: 1
Оцінка Q: 0.7036
Вибіркова дисперсія: 0.20858875775155034
Асимптотичний довірчий інтервал: [0.6869629038602472, 0.7202370961397528]
Кількість спостережень: 5000
N*: 27937.896419534052

Кількість зразків: 10
Оцінка Q: 0.1636
Вибіркова дисперсія: 0.1368624124824965
Асимптотичний довірчий інтервал: [0.15012359131379818, 0.1770764086862018]
Кількість спостережень: 5000
N*: 339056.4879479564

Кількість зразків: 100
Оцінка Q: 0.0152
Вибіркова дисперсія: 0.014971954390878176
Асимптотичний довірчий інтервал: [0.01074270554486938, 0.01965729445513062]
Кількість спостережень: 5000
N*: 4296806.400753835

Кількість зразків: 1000
Оцінка Q: 0.0016
Вибіркова дисперсія: 0.001597759551910382
Асимптотичний довірчий інтервал: [0.00014391144361205945, 0.0030560885563879407]
Кількість спостережень: 5000
N*: 41383376.675335065



Метод 2

In [16]:

m_arr = [1, 10, 100, 1_000, 10_000]  # for calculating Q_m
n_0 = 100
q_2 = lambda samples: 1 / (1 + np.sum(samples))

for m in m_arr:
    indicators = []
    for j in range(n_0):
        samples = np.random.random(m)
        indicators.append(q_2(samples))
    Q = Q_m(indicators, len(indicators))
    sigma = calc_variance(indicators, len(indicators), Q)
    print(f"Кількість зразків: {m}")
    print(f"Оцінка Q: {Q}")
    print(f"Вибіркова дисперсія: {sigma}")
    print(f"Асимптотичний довірчий інтервал: {calculate_ci_2(indicators, gama)}")
    print(f"Кількість спостережень: {n_0}")
    print(f"N*: {(2.575 * 100) ** 2 * sigma / (Q ** 2)}\n")

Кількість зразків: 1
Оцінка Q: 0.6909606530548138
Вибіркова дисперсія: 0.021020157331772215
Асимптотичний довірчий інтервал: [0.6536154297413147, 0.728305876368312]
Кількість спостережень: 100
N*: 2919.3340647790487

Кількість зразків: 10
Оцінка Q: 0.17217434045309546
Вибіркова дисперсія: 0.0007247906732450595
Асимптотичний довірчий інтервал: [0.16523970911872327, 0.17910897178746749]
Кількість спостережень: 100
N*: 1621.1761532446276

Кількість зразків: 100
Оцінка Q: 0.019578885293872774
Вибіркова дисперсія: 8.625088759545349e-07
Асимптотичний довірчий інтервал: [0.019339664533707175, 0.019818106054038366]
Кількість спостережень: 100
N*: 149.19082508118805

Кількість зразків: 1000
Оцінка Q: 0.0019965912736311995
Вибіркова дисперсія: 1.0334469480729866e-09
Асимптотичний довірчий інтервал: [0.0019883106857033427, 0.0020048718615590564]
Кількість спостережень: 100
N*: 17.189542438635527

Кількість зразків: 10000
Оцінка Q: 0.00019995887504449156
Вибіркова дисперсія: 1.2673912987417323e-12

Метод 4

In [17]:
m_arr = [1, 10, 100, 1_000, 10_000]  # for calculation Q_m
n_0 = 5
q_4 = lambda m, x, samples: (1 / (m - 1)) * ((np.sum(samples)) / (1 + np.sum(samples)))

for m in m_arr:
    indicators = []
    for j in range(n_0):
        samples = np.random.random(m)
        indicators.append(q_2(samples))

    Q = Q_m(indicators, len(indicators))
    sigma = calc_variance(indicators, len(indicators), Q)

    print(f"Кількість зразків: {m}")
    print(f"Оцінка Q: {Q}")
    print(f"Вибіркова дисперсія: {sigma}")
    print(f"Асимптотичний довірчий інтервал: {calculate_ci_2(indicators, gama)}")
    print(f"Кількість спостережень: {n_0}")
    print(f"N*: {(2.575 * 100) ** 2 * sigma / (Q ** 2)}\n")

Кількість зразків: 1
Оцінка Q: 0.6763812779654477
Вибіркова дисперсія: 0.02257625255644735
Асимптотичний довірчий інтервал: [0.5032968469206133, 0.8494657090102821]
Кількість спостережень: 5
N*: 3272.0743670406237

Кількість зразків: 10
Оцінка Q: 0.1847351312375894
Вибіркова дисперсія: 0.0003790521134224706
Асимптотичний довірчий інтервал: [0.16230759646911017, 0.2071626660060686]
Кількість спостережень: 5
N*: 736.4688835924711

Кількість зразків: 100
Оцінка Q: 0.01970261854829894
Вибіркова дисперсія: 1.3675849866464198e-06
Асимптотичний довірчий інтервал: [0.01835548938006688, 0.021049747716531]
Кількість спостережень: 5
N*: 233.5935746410745

Кількість зразків: 1000
Оцінка Q: 0.0019923615653446797
Вибіркова дисперсія: 4.049320668294233e-09
Асимптотичний довірчий інтервал: [0.0019190583267552329, 0.0020656648039341263]
Кількість спостережень: 5
N*: 67.63949034959347

Кількість зразків: 10000
Оцінка Q: 0.00020028756806775625
Вибіркова дисперсія: 2.5191353785154504e-12
Асимптотичний дов